Define data.

In [6]:
J = ['peach', 'raspberry']
profit = {'peach': 8, 'raspberry': 12}
sugarReq = {'peach': 4, 'raspberry': 3}
juiceReq = {'peach': 1, 'raspberry': 2}
demand = {'peach': 4, 'raspberry': 2}
juiceAvail = 5
sugarAvail =12 

Import Pyomo and declare a model object.

In [7]:
from pyomo.environ import *

model = ConcreteModel()

Declare nonnegative variables for peach and raspberry lemonade.

In [8]:
try: 
    model.del_component(model.x)
    model.del_component(model.x_index)
except:
    print("Defining variables for first time")
model.x = Var(J, domain=NonNegativeReals)


Defining variables for first time


Define the objective function.

In [9]:
try:
    model.del_component(model.Profit)
except:
    print("Defining objective for the first time")
model.Profit = Objective(expr = sum(profit[i]*model.x[i] for i in J), sense=maximize)

Defining objective for the first time


Specify the juice constraint. 

In [10]:
try:
    model.del_component(model.JuiceConstraint)
except:
    print("Defining JuiceConstraint for the first time")
model.JuiceConstraint = Constraint(expr = sum(juiceReq[i]*model.x[i] for i in J) <= juiceAvail)

Defining JuiceConstraint for the first time


Specify the sugar constraint.

In [11]:
try:
    model.del_component(model.SugarConstraint)
except:
    print("Defining SugarConstraint for the first time")
model.SugarConstraint = Constraint(expr = sum(sugarReq[i]*model.x[i] for i in J) <= sugarAvail)

Defining SugarConstraint for the first time


In [12]:
model.pprint()

1 Set Declarations
    x_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['peach', 'raspberry']

1 Var Declarations
    x : Size=2, Index=x_index
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
            peach :     0 :  None :  None : False :  True : NonNegativeReals
        raspberry :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    Profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[peach] + 12*x[raspberry]

2 Constraint Declarations
    JuiceConstraint : Size=1, Index=None, Active=True
        Key  : Lower : Body                      : Upper : Active
        None :  -Inf : x[peach] + 2*x[raspberry] :   5.0 :   True
    SugarConstraint : Size=1, Index=None, Active=True
        Key  : Lower : Body                        : Upper : Active
        None :  -Inf : 4*x[peach] + 3*x[raspberry] :  12.0 :   True

5 Declarations: 

Add the demand constraints as a set of constraints.  Define the list of constraints in a separate chunk.

In [13]:
try:
    model.del_component(model.Demand)
    model.del_component(model.Demand_index)
except:
    print("Defining demand constraints for the first time)")
model.Demand = ConstraintList()

Defining demand constraints for the first time)


Now add a constraint for each lemonade.

In [14]:
for i in J:
    model.Demand.add(model.x[i] <= demand[i])

Specify solver and solve.


In [15]:

solver = SolverFactory('glpk')
status = solver.solve(model)

Get status.

In [16]:
print('Status = %s' % status)

Status = 
Problem: 
- Name: unknown
  Lower bound: 33.6
  Upper bound: 33.6
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 3
  Number of nonzeros: 7
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.013503313064575195
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



Get solution and objective function value.

In [17]:
for i in J:
    print('%s = %f' % (model.x[i], value(model.x[i])))
print("Objective = %f" % value(model.Profit))

x[peach] = 1.800000
x[raspberry] = 1.600000
Objective = 33.600000


The optimal solution is to produce 1.8 gallons of peach lemonade and 1.5 gallons of raspberry lemonade for a profit of \$33.60.